# **Arms Data: ETL**

## Objectives

* Evaluate size of data set
* Check data types
* Normalise values and columns
* Prepare features for K-means clustering 

## Inputs

* This ETL notebook requires the trade-register.csv dataset obtainable from this link: https://armstransfers.sipri.org/ArmsTransfer/TransferRegister
* Once downloaded please save the file into data_folder/Raw 
* DISCLAIMER: SIPRI's terms of fair use forbid the reproduction of over 10% of their datasets
    - DO NOT publish either the raw or transformed dataset as this would be a breach of fair use policy
    - Please keep any artifacts created on your local machine
## Outputs

* By the end of the notebook a transformed dataset will be loaded into the Processed sub-folder 




---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\jackr\\OneDrive\\Desktop\\my_projects\\Capstone\\Arms-Import-Export-Analysis\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\jackr\\OneDrive\\Desktop\\my_projects\\Capstone\\Arms-Import-Export-Analysis'

In [4]:
print(os.listdir())


['.git', '.gitignore', '.python-version', '.slugignore', '.venv', 'data_folder', 'jupyter_notebooks', 'Procfile', 'README.md', 'requirements.txt', 'setup.sh']


# Cleaning and normalising 

In this section the focus will be on transforming the existing data as opposed to feature engineering.

First the necessary libraries will be imported.

In [5]:
import pandas as pd

Now the CSV file containing the dataset can be parsed into a DataFrame.

In [ ]:
arms_df = pd.read_csv('data_folder/Raw/trade-register.csv')
arms_df.head()

,Recipient,Supplier,Year of order,,Number ordered,.1,Weapon designation,Weapon description,Number delivered,.2,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
0,Afghanistan,Russia,2002.0,NaN,3.0,NaN,Mi-17,transport helicopter,3.0,NaN,2002,Second hand,Second-hand; aid,2.90,8.70,8.70
1,Afghanistan,Turkiye,2007.0,NaN,24.0,NaN,M-114 155mm,towed gun,24.0,NaN,2007,Second hand,Second-hand; aid,0.20,4.80,4.80
2,Afghanistan,United States,2004.0,?,188.0,?,M-113,armoured personnel carrier,188.0,?,2005,Second hand,Second-hand; aid; M-113A2 version; incl 15 M-5...,0.10,18.80,18.80
3,Afghanistan,United States,2016.0,NaN,53.0,NaN,S-70 Black Hawk,transport helicopter,53.0,?,2017; 2018; 2019; 2020,Second hand but modernized,Second-hand UH-60A modernized to UH-60A+ befor...,4.29,227.37,227.37
4,Afghanistan,Soviet Union,1973.0,?,100.0,?,T-62,tank,100.0,?,1975; 1976,New,NaN,1.80,180.00,180.00


It is worth noting that this dataset measures each order by 'TIV' (Trend Indicator Value), this is SIPRI's way of calculating military capability and is used in place of the total market value.

First the structure and data types will be checked.

In [7]:
arms_df.shape

(29507, 16)

At just under 30,000 rows this dataset will be excellent for analysis.

In [8]:
arms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29507 entries, 0 to 29506
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Recipient                       29503 non-null  object 
 1   Supplier                        29507 non-null  object 
 2   Year of order                   29507 non-null  float64
 3                                   15574 non-null  object 
 4   Number ordered                  29217 non-null  float64
 5    .1                             12297 non-null  object 
 6   Weapon designation              29503 non-null  object 
 7   Weapon description              29503 non-null  object 
 8   Number delivered                29503 non-null  float64
 9    .2                             16515 non-null  object 
 10  Year(s) of delivery             28348 non-null  object 
 11  status                          29503 non-null  object 
 12  Comments                        

Looks like the Year of order values need to be converted to datetime, as there are often multiple years in the Year(s) of delivery column different action may need to be taken.

In [9]:
arms_df.describe()

,Year of order,Number ordered,Number delivered,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
count,29507.000000,29217.000000,29503.000000,29499.000000,29499.000000,29499.000000
mean,1989.283792,124.013896,122.131071,7.553335,76.846316,70.923255
std,31.289445,806.611909,837.095692,27.676713,264.162104,245.602120
min,0.360000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1973.000000,3.000000,3.000000,0.240000,4.400000,3.700000
50%,1988.000000,10.000000,10.000000,1.000000,15.000000,14.000000
75%,2009.000000,50.000000,43.000000,5.000000,50.000000,47.000000
max,2024.000000,50000.000000,50000.000000,1250.000000,10117.500000,10117.500000


The values shown above will provide a good benchmark for exploratory analysis.

Now the number of duplicated rows and null values are displayed.

In [10]:
arms_df.duplicated().sum()


9

In [11]:
arms_df.isnull().sum()

Recipient                             4
Supplier                              0
Year of order                         0
                                  13933
Number ordered                      290
 .1                               17210
Weapon designation                    4
Weapon description                    4
Number delivered                      4
 .2                               12992
Year(s) of delivery                1159
status                                4
Comments                           5296
SIPRI TIV per unit                    8
SIPRI TIV for total order             8
SIPRI TIV of delivered weapons        8
dtype: int64

Duplicated values should be removed and it looks like dropping the years of delivery column would be best due to its unpredictable format and many missing values.

There are some unlabeled columns with unknown values which will be dropped .

In [12]:
# dropping columns by indices
arms_df = arms_df.drop(columns=[arms_df.columns[9], 
                                arms_df.columns[5], 
                                arms_df.columns[3]]
)
arms_df.head()


,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
0,Afghanistan,Russia,2002.0,3.0,Mi-17,transport helicopter,3.0,2002,Second hand,Second-hand; aid,2.90,8.70,8.70
1,Afghanistan,Turkiye,2007.0,24.0,M-114 155mm,towed gun,24.0,2007,Second hand,Second-hand; aid,0.20,4.80,4.80
2,Afghanistan,United States,2004.0,188.0,M-113,armoured personnel carrier,188.0,2005,Second hand,Second-hand; aid; M-113A2 version; incl 15 M-5...,0.10,18.80,18.80
3,Afghanistan,United States,2016.0,53.0,S-70 Black Hawk,transport helicopter,53.0,2017; 2018; 2019; 2020,Second hand but modernized,Second-hand UH-60A modernized to UH-60A+ befor...,4.29,227.37,227.37
4,Afghanistan,Soviet Union,1973.0,100.0,T-62,tank,100.0,1975; 1976,New,NaN,1.80,180.00,180.00


Following on from this the years of delivery column can aslo be dropped.

In [13]:
# drop Year(s) of delivery column
arms_df = arms_df.drop(columns=[arms_df.columns[7]])
arms_df.head()




,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
0,Afghanistan,Russia,2002.0,3.0,Mi-17,transport helicopter,3.0,Second hand,Second-hand; aid,2.90,8.70,8.70
1,Afghanistan,Turkiye,2007.0,24.0,M-114 155mm,towed gun,24.0,Second hand,Second-hand; aid,0.20,4.80,4.80
2,Afghanistan,United States,2004.0,188.0,M-113,armoured personnel carrier,188.0,Second hand,Second-hand; aid; M-113A2 version; incl 15 M-5...,0.10,18.80,18.80
3,Afghanistan,United States,2016.0,53.0,S-70 Black Hawk,transport helicopter,53.0,Second hand but modernized,Second-hand UH-60A modernized to UH-60A+ befor...,4.29,227.37,227.37
4,Afghanistan,Soviet Union,1973.0,100.0,T-62,tank,100.0,New,NaN,1.80,180.00,180.00


Now the NaN values in the comments column should be addressed, since there are a large number of NaN values it would be detrimental to drop all rows that contain these values. It would also be unfortunate to lose the whole comments column as it could provide some valuable insights in a dashboard.

The third option is to replace these values with 'No additional info' as this is a good description of what the NaN values connote.

In [14]:
# replace NaN values in comments column with 'No additional info'
arms_df['Comments'] = arms_df['Comments'].fillna('No additional info')
arms_df.head()

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
0,Afghanistan,Russia,2002.0,3.0,Mi-17,transport helicopter,3.0,Second hand,Second-hand; aid,2.90,8.70,8.70
1,Afghanistan,Turkiye,2007.0,24.0,M-114 155mm,towed gun,24.0,Second hand,Second-hand; aid,0.20,4.80,4.80
2,Afghanistan,United States,2004.0,188.0,M-113,armoured personnel carrier,188.0,Second hand,Second-hand; aid; M-113A2 version; incl 15 M-5...,0.10,18.80,18.80
3,Afghanistan,United States,2016.0,53.0,S-70 Black Hawk,transport helicopter,53.0,Second hand but modernized,Second-hand UH-60A modernized to UH-60A+ befor...,4.29,227.37,227.37
4,Afghanistan,Soviet Union,1973.0,100.0,T-62,tank,100.0,New,No additional info,1.80,180.00,180.00


Now that a significant portion of the NaN values in the whole dataset have been resolved I feel comfortable dropping any remaining rows containing NaN values as they will have a negligible impact if removed.

In [15]:
# drop any remaining rows with NaN values
arms_df = arms_df.dropna()
arms_df.isnull().sum()

Recipient                         0
Supplier                          0
Year of order                     0
Number ordered                    0
Weapon designation                0
Weapon description                0
Number delivered                  0
status                            0
Comments                          0
SIPRI TIV per unit                0
SIPRI TIV for total order         0
SIPRI TIV of delivered weapons    0
dtype: int64

Duplicate rows will also be dropped, there's a very low likelihood that these duplicates represent unique entries as the 'comments' column often includes distinct free text entries that are identical between the duplicate rows and the original rows.

In [16]:
# dataframe containing only duplicate rows paired with their original rows
duplicate_rows = arms_df[arms_df.duplicated(keep=False)]
duplicate_rows.head(10)

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
2293,Belarus,Russia,2015.0,4.0,Yak-130,trainer/combat aircraft,4.0,New,No additional info,10.50,42.00,42.00
2313,Belarus,Russia,2015.0,4.0,Yak-130,trainer/combat aircraft,4.0,New,No additional info,10.50,42.00,42.00
2782,Botswana,United States,2018.0,1.0,C-212,transport aircraft,1.0,Second hand,Second-hand,1.52,1.52,1.52
2783,Botswana,United States,2018.0,1.0,C-212,transport aircraft,1.0,Second hand,Second-hand,1.52,1.52,1.52
4414,Chile,United Kingdom,1986.0,10.0,MM-38 Exocet,anti-ship missile,10.0,Second hand,Second-hand; for County (Prat) destroyer,0.30,3.00,3.00
4420,Chile,United Kingdom,1986.0,10.0,MM-38 Exocet,anti-ship missile,10.0,Second hand,Second-hand; for County (Prat) destroyer,0.30,3.00,3.00
4451,Chile,United States,1971.0,1.0,C-130H Hercules,transport aircraft,1.0,New,No additional info,28.00,28.00,28.00
4455,Chile,United States,1971.0,1.0,C-130H Hercules,transport aircraft,1.0,New,No additional info,28.00,28.00,28.00
4592,Chile,United States,2000.0,1.0,Bell-412,helicopter,1.0,Second hand,Second-hand,2.00,2.00,2.00
4607,Chile,United States,2000.0,1.0,Bell-412,helicopter,1.0,Second hand,Second-hand,2.00,2.00,2.00


In [17]:
# drop duplicated rows
arms_df = arms_df.drop_duplicates()
arms_df.duplicated().sum()


0

---

# Feature Engineering

In this shorter section new features will be created as well as a new, more comprehensive index to prepare the data for EDA and clustering later on

In [18]:
# Create datetime and transaction ID
arms_df['OrderDate'] = pd.to_datetime(arms_df['Year of order'], format='%Y')
arms_df['TransactionID'] = arms_df['Supplier'] + '_' + arms_df['Year of order'].astype(str) + '_' + arms_df.index.astype(str)

# Add features for clustering
arms_df['Supplier_Total_TIV'] = arms_df.groupby('Supplier')['SIPRI TIV for total order'].transform('sum')
arms_df['Recipient_Total_TIV'] = arms_df.groupby('Recipient')['SIPRI TIV for total order'].transform('sum')
arms_df['Trade_Volume'] = arms_df.groupby(['Supplier', 'Recipient'])['SIPRI TIV for total order'].transform('count')

# Optional: Add temporal features
arms_df['Decade'] = (arms_df['Year of order'] // 10) * 10
arms_df['Trade_Frequency'] = arms_df.groupby(['Supplier', 'Recipient', 'Decade'])['SIPRI TIV for total order'].transform('count')

# Set multi-index
arms_df = arms_df.set_index(['OrderDate', 'TransactionID'])
arms_df.head()

,,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Supplier_Total_TIV,Recipient_Total_TIV,Trade_Volume,Decade,Trade_Frequency
OrderDate,TransactionID,,,,,,,,,,,,,,,,,
2002-01-01,Russia_2002.0_0,Afghanistan,Russia,2002.0,3.0,Mi-17,transport helicopter,3.0,Second hand,Second-hand; aid,2.90,8.70,8.70,165817.26,15809.14,11,2000.0,6
2007-01-01,Turkiye_2007.0_1,Afghanistan,Turkiye,2007.0,24.0,M-114 155mm,towed gun,24.0,Second hand,Second-hand; aid,0.20,4.80,4.80,6689.54,15809.14,1,2000.0,1
2004-01-01,United States_2004.0_2,Afghanistan,United States,2004.0,188.0,M-113,armoured personnel carrier,188.0,Second hand,Second-hand; aid; M-113A2 version; incl 15 M-5...,0.10,18.80,18.80,849588.93,15809.14,34,2000.0,3
2016-01-01,United States_2016.0_3,Afghanistan,United States,2016.0,53.0,S-70 Black Hawk,transport helicopter,53.0,Second hand but modernized,Second-hand UH-60A modernized to UH-60A+ befor...,4.29,227.37,227.37,849588.93,15809.14,34,2010.0,31
1973-01-01,Soviet Union_1973.0_4,Afghanistan,Soviet Union,1973.0,100.0,T-62,tank,100.0,New,No additional info,1.80,180.00,180.00,452121.07,15809.14,82,1970.0,25


A multi-level index structure has been created using OrderDate (from Year of order) and TransactionID (unique identifier combining supplier, year and row index) to enable both temporal analysis and unique transaction tracking. 

New features were added to help with clustering: total TIV values for suppliers and recipients show overall trade volumes, trade volume counts show relationship strength between countries, and temporal features group trades by decades to show trading patterns over time. 

These features will help identify meaningful clusters in the arms trade data and enable analysis of trading relationships over time.

The new datetime formatted 'OrderDate' index renders the Year of order column obsolete so it can now be dropped.

In [19]:
# drop year of order column
arms_df = arms_df.drop(columns=['Year of order'])
arms_df.head()

,,Recipient,Supplier,Number ordered,Weapon designation,Weapon description,Number delivered,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Supplier_Total_TIV,Recipient_Total_TIV,Trade_Volume,Decade,Trade_Frequency
OrderDate,TransactionID,,,,,,,,,,,,,,,,
2002-01-01,Russia_2002.0_0,Afghanistan,Russia,3.0,Mi-17,transport helicopter,3.0,Second hand,Second-hand; aid,2.90,8.70,8.70,165817.26,15809.14,11,2000.0,6
2007-01-01,Turkiye_2007.0_1,Afghanistan,Turkiye,24.0,M-114 155mm,towed gun,24.0,Second hand,Second-hand; aid,0.20,4.80,4.80,6689.54,15809.14,1,2000.0,1
2004-01-01,United States_2004.0_2,Afghanistan,United States,188.0,M-113,armoured personnel carrier,188.0,Second hand,Second-hand; aid; M-113A2 version; incl 15 M-5...,0.10,18.80,18.80,849588.93,15809.14,34,2000.0,3
2016-01-01,United States_2016.0_3,Afghanistan,United States,53.0,S-70 Black Hawk,transport helicopter,53.0,Second hand but modernized,Second-hand UH-60A modernized to UH-60A+ befor...,4.29,227.37,227.37,849588.93,15809.14,34,2010.0,31
1973-01-01,Soviet Union_1973.0_4,Afghanistan,Soviet Union,100.0,T-62,tank,100.0,New,No additional info,1.80,180.00,180.00,452121.07,15809.14,82,1970.0,25


It's time now to load our new cleaned dataset with additional features into the designated folder

In [21]:
arms_df.to_csv('data_folder/Processed/arms_trade_cleaned.csv')

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Conclusions

* A thorough ETL process was conducted 
* The integrity of the data was preserved throughout the process
* Compliance with SIPRI's fair use policy has been maintained 
* New and relevant features have been added to enhance the data for EDA and clustering purposes

Now that the ETL process has been completed the enhanced dataset can be explored in more depth with python visualisations. To reflect, the dataset provided a good challenge for the ETL phase. The dataset was comprehensive and well structured but not without its faults with many null values and some duplicates. On top of this some void columns had to be removed. The features created in this section will save valuable time when tackling the visualisation and clustering stages.